## Initial imports

In [1]:
import os
import math
import random
import numpy as np
from collections import defaultdict
import torch
from torchvision.ops import nms
from PIL import Image
import matplotlib.pyplot as plt
import pycocotools
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Code from Maastricht to access GPU of choice

import torch
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [3]:
torch.cuda.set_device(0)
device = torch.device(f'cuda:{0}' if torch.cuda.is_available() else 'cpu')

In [4]:
torch.cuda.current_device(), torch.cuda.get_device_name()

(0, 'GeForce GTX 1080 Ti')

## Load data

In [5]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode


# Class for a customized dataset
# In this case preprocessed CEM images combined in a 3-channel RGB .jpg format
# and the corresponding mask of present lesions in a 1-channel .png format
class CustomImageDataset(Dataset):
    def __init__(self, root, annotations_file, img_dir, mask_dir, train=False, transform=None, target_transform=None):
        # Read the .csv file with all the information
        self.img_labels = pd.read_csv(os.path.join(root, annotations_file))
        # Define the directories of the images and masks
        self.img_dir = os.path.join(root, img_dir)
        self.mask_dir = os.path.join(root, mask_dir)
        # Define whethet transformations are included
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        # Return the number of cases in the dataset
        # In this set, CC and MLO of the same breast are considered different cases
        return len(self.img_labels)

    def __getitem__(self, idx):
        # Read the image and the mask for a case from the directories
        img_path = self.img_labels.iloc[idx, 0]
        mask_path = self.img_labels.iloc[idx,6]
        img_path = img_path.replace('E:', '\\\\tsclient\E')
        mask_path = mask_path.replace('E:', '\\\\tsclient\E')
        image = read_image(img_path).float()
        mask = read_image(mask_path)
        
        xmin = self.img_labels.iloc[idx, 1]
        xmax = self.img_labels.iloc[idx, 2]
        ymin = self.img_labels.iloc[idx, 3]
        ymax = self.img_labels.iloc[idx, 4]
        
        # Apply transformations if defined
        flipint = random.random()
        if self.train and flipint > 0.5 :        
            image = T.RandomHorizontalFlip(p=1.0)(image)
            mask = T.RandomHorizontalFlip(p=1.0)(mask)
            
            ymax_temp = image.shape[0] - ymin
            ymin = image.shape[0] - ymax
            ymax = ymax_temp

        # Resize so all images and masks have the same size
#         image = T.Resize([800,800])(image)
#         mask = T.Resize([800,800])(mask)    
#         resize_scale_x = 800/image.size()[1]
#         resize_scale_y = 800/image.size()[2]

        # Resize if necessary
        # First the smallest dimension is reduced to 400 if it is larger
        # Then the largest dimension is reduced to 650 if it is still larger
        resize_scale = 1.0
        min_size_idx = np.argmin([image.size()[1], image.size()[2]])
        
        if min_size_idx == 0 and image.size()[1] > 400 :
            resize_scale *= 400/image.size()[1]
            image = T.Resize([400, int(400*image.size()[2]/image.size()[1])])(image)
            mask = T.Resize([400, int(400*image.size()[2]/image.size()[1])], interpolation=InterpolationMode.NEAREST)(mask)
        if min_size_idx == 0 and image.size()[2] > 650 :
            resize_scale *= 650/image.size()[2]
            image = T.Resize([int(650*image.size()[1]/image.size()[2]), 650])(image)
            mask = T.Resize([int(650*image.size()[1]/image.size()[2]), 650], interpolation=InterpolationMode.NEAREST)(mask)
        
        if min_size_idx == 1 and image.size()[2] > 400 :
            resize_scale *= 400/image.size()[2]
            image = T.Resize([int(400*image.size()[1]/image.size()[2]), 400])(image)
            mask = T.Resize([int(400*image.size()[1]/image.size()[2]), 400], interpolation=InterpolationMode.NEAREST)(mask)
        if min_size_idx == 1 and image.size()[1] > 650 :
            resize_scale *= 650/image.size()[1]
            image = T.Resize([650, int(650*image.size()[2]/image.size()[1])])(image)
            mask = T.Resize([650, int(650*image.size()[2]/image.size()[1])], interpolation=InterpolationMode.NEAREST)(mask)
            
        # Normalize image with mean and standard deviation per channel
        mean = torch.mean(image, dim=(1,2))
        stdev = torch.std(image, dim=(1,2))
        image = T.Normalize(mean, stdev)(image)
        
        # Rescale to [0,1] range per channel
        for dim in range(3) :
            image[dim] -= torch.min(image[dim])
            image[dim] /= torch.max(image[dim])  
        
        # Create separate channel in mask for each lesion
        mask_out = np.zeros((torch.max(mask).item(), mask.shape[-2], mask.shape[-1]))
        for lesion_idx in range(torch.max(mask).item()) :
#             mask_out[lesion_idx][mask[0]==lesion_idx+1] = 1  
            mask_out[lesion_idx][mask[0]>0] = 1 # alternative for wrong masks with only one lesion
        
        # Read the location of the lesion bounding box from the .csv file
        boxes = [[ymin*resize_scale, xmin*resize_scale, ymax*resize_scale, xmax*resize_scale]]
        # Read the label of the lesion from the .csv file
        labels = self.img_labels.iloc[idx, 5]  + 1   # 0 represents background class, thus 1=benign, 2=malignant
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        labels = torch.tensor([labels])
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        mask_out = torch.from_numpy(mask_out)
        mask_out = mask_out.to(torch.uint8)
        
        iscrowd = torch.zeros((2,), dtype=torch.int64)  
            
        target = {}
        target["image_id"] = image_id
        target["masks"] = mask_out
        target["boxes"] = boxes
        target["area"] = area
        target["labels"] = labels
        target["iscrowd"] = iscrowd
            
        return image, target


In [6]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode

from monai.transforms import CropForeground, RandSpatialCropSamplesd, Flipd, RandCropByPosNegLabeld, Resized, FillHoles, NormalizeIntensityd
from monai.utils.enums import InterpolateMode

# Define the transformations of Monai to be used later on
resize_fcn_large = Resized(["image","mask"],
                     spatial_size=(1300,800),
                     mode=[InterpolateMode.BILINEAR,InterpolateMode.NEAREST])
resize_fcn_small = Resized(["image","mask"],
                     spatial_size=(650,400),
                     mode=[InterpolateMode.BILINEAR,InterpolateMode.NEAREST])
# crops_fcn = RandSpatialCropSamplesd(["image","mask"],
#                                     num_samples=10,
#                                     roi_size=(650,400),
#                                     random_size=False)
crop_fcn_small = RandCropByPosNegLabeld(["image", "mask"],
                                  "mask",
                                  (650,400),
                                  pos=0.1, neg=1.0,
                                  num_samples=10)
flip_fcn = Flipd(["image","mask"],
                 spatial_axis=1)
fill_fcn = FillHoles()
norm_fcn = NormalizeIntensityd(["image"], channel_wise=True, nonzero=False)

# Class for a customized dataset
# In this case preprocessed CEM images combined in a 3-channel RGB .jpg format
# and the corresponding mask of present lesions in a 1-channel .png format
class CustomImageDatasetMonai(Dataset):
    def __init__(self, root, annotations_file, img_dir, mask_dir, train=False, transform=None, target_transform=None):
        # Read the .csv file with all the information
        self.img_labels = pd.read_csv(os.path.join(root, annotations_file))
        # Define the directories of the images and masks
        self.img_dir = os.path.join(root, img_dir)
        self.mask_dir = os.path.join(root, mask_dir)
        # Define whether transformations are included
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        # Return the number of cases in the dataset
        # In this set, CC and MLO of the same breast are considered different cases
        
        return len(self.img_labels)

    def __getitem__(self, idx):
        
        # Read the image and the mask for a case from the directories
        img_path = self.img_labels.iloc[idx, 0]
        mask_path = self.img_labels.iloc[idx,6]
        image = read_image(img_path).float()
        mask = read_image(mask_path)
        
        print('patient', idx, img_path)
        
        init_dict = {}
        init_dict["image"] = image
        init_dict["mask"] = mask
#         print('Init', init_dict["mask"].shape, np.unique(init_dict["mask"],return_counts=True))
        
#         # Resize the image to 1300 by 800
#         init_dict = resize_fcn(init_dict)
        
        # Create 10 samples of croppped image and mask
#         crop_dict = crops_fcn(init_dict)
#         print(crop_dict[0]["image"].shape)
        # Find the first sample with a nonzero mask
    
        crop_fcn_large = RandCropByPosNegLabeld(["image", "mask"],
                                                "mask",
                                                (int(mask.shape[1]/2),int(mask.shape[2]/2)),
                                                pos=0.1, neg=1.0,
                                                num_samples=10)
    
        # Generate 10 croppped samples until a mask with the complete lesion is obtained
        max_pos = 0
        max_pos_idx = 0
        while max_pos == 0 :
            crop_dict = crop_fcn_large(init_dict)
            for crop_idx in range(10) :
#                 print(crop_idx, np.count_nonzero(crop_dict[crop_idx]["mask"]), crop_dict[crop_idx]["mask"].shape)
                if np.count_nonzero(crop_dict[crop_idx]["mask"]) > 0.75*np.count_nonzero(mask) :
                    max_pos = np.count_nonzero(crop_dict[crop_idx]["mask"])
                    max_pos_idx = crop_idx
            
#         print('Crop', crop_dict[max_pos_idx]["mask"].shape, np.unique(crop_dict[max_pos_idx]["mask"],return_counts=True))
        
        resize_dict = resize_fcn_small(crop_dict[max_pos_idx])
#         print('Resize', resize_dict["mask"].shape, np.unique(resize_dict["mask"],return_counts=True))
        
        # Data augmentation by horizontal flipping
        flipint = random.random()
        if self.train and flipint > 0.5 : 
            resize_dict = flip_fcn(resize_dict)
        else :
            resize_dict = resize_dict
#         print('Flip', resize_dict["mask"].shape, np.unique(resize_dict["mask"],return_counts=True))
            
        # Normalize image as values should be in [0,1]
        min_vals = [torch.min(resize_dict["image"][0]).item(), torch.min(resize_dict["image"][1]).item(), torch.min(resize_dict["image"][2]).item()]
        max_vals = [torch.max(resize_dict["image"][0]).item(), torch.max(resize_dict["image"][1]).item(), torch.max(resize_dict["image"][2]).item()]
        norm_minmax_fcn = NormalizeIntensityd(["image"],
                                              min_vals, [a_i-b_i for a_i,b_i in zip(max_vals,min_vals)],
                                              channel_wise=True, nonzero=False)
        
#         resize_dict = norm_fcn(resize_dict)
        resize_dict = norm_minmax_fcn(resize_dict)
#         print('Resize', resize_dict["mask"].shape, np.unique(resize_dict["mask"],return_counts=True), np.unique(resize_dict["image"]))
                        
        mask_out = np.zeros((1, resize_dict["mask"].shape[-2], resize_dict["mask"].shape[-1]))
        mask_out[0][resize_dict["mask"][0]>0] = 1 # alternative for wrong masks with only one lesion
#         print('Mask out', mask_out.shape, np.unique(mask_out,return_counts=True))
            
        # Find the new bounding box on the transformed image
        bbox = CropForeground().compute_bounding_box(resize_dict["mask"])
#         print(bbox)
        bbox_vals = [[bbox[0][1], bbox[0][0], bbox[1][1], bbox[1][0]]]
        area_vals = [(bbox_vals[0][3] - bbox_vals[0][1]) * (bbox_vals[0][2] - bbox_vals[0][0])]      
        labels = self.img_labels.iloc[idx, 5]  + 1   # 0 represents background class, thus 1=benign, 2=malignant
        iscrowd = torch.zeros((2,), dtype=torch.int64)  
#         print(area_vals)
            
        # Create the target dictionary
        target = {}
        target["image_id"] = torch.tensor([idx])
        target["masks"] = torch.from_numpy(mask_out).to(torch.uint8)
        target["boxes"] = torch.as_tensor(bbox_vals, dtype=torch.float32)
        target["area"] = torch.as_tensor(area_vals, dtype=torch.float32)
        target["labels"] = torch.tensor([torch.as_tensor(labels, dtype=torch.int64)])
        target["iscrowd"] = iscrowd
        
        return resize_dict["image"], target
    

    def lenclasses(self):
        
        lenben = 0
        lenmal = 0
        
        for idx in range(len(self.img_labels)) :
            labels = self.img_labels.iloc[idx, 5]
            
            if labels == 0 :
                lenben += 1
            elif labels == 1 :
                lenmal += 1
        
        return lenben, lenmal

In [7]:
# testdatadir = 'B:\\Astrid\\Preprocessed\\TrainCalcClusterSynthetic'
# test_data = CustomImageDataset(testdatadir, 'annotations_test_calccluster_synthetic_onlymaldef.csv', 'colored_to_jpg', 'mask_to_png')
# test_data = CustomImageDataset(testdatadir, 'annotations_val_calccluster_synthetic_onlymaldef.csv', 'colored_to_jpg', 'mask_to_png')

# testdatadir = 'B:\\Astrid\\Preprocessed\\TrainCalcClusterReal'
# test_data = CustomImageDataset(testdatadir, 'annotations_train_calccluster_real_onlymal.csv', 'colored_to_jpg', 'mask_to_png')

# testdatadir = 'B:\\Astrid\\Preprocessed\\TestCalcClusterSynthetic_SameImage'
# test_data = CustomImageDataset(testdatadir, 'annotations_test_calccluster_synthetic.csv', 'colored_to_jpg', 'mask_to_png')

# testdatadir = 'B:\\Astrid\\Preprocessed\\TestCalcClusterSynthetic_SameCluster'
# test_data = CustomImageDataset(testdatadir, 'annotations_test_calccluster_synthetic.csv', 'colored_to_jpg', 'mask_to_png')

# testdatadir = 'B:\\Astrid\\Preprocessed\\TestCalcClusterReal'
# test_data = CustomImageDataset(testdatadir, 'annotations_test_calccluster_real_onlymal.csv', 'colored_to_jpg', 'mask_to_png')
# test_data_short = []
# for idx in range(16) :
#     test_data_short.append(test_data[idx])
    
# savedir = 'B:\\Astrid\\Preprocessed\\Models'
    
# traindatadir = 'B:\\Astrid\\Preprocessed\\train_set_preprocessedcalc'
# trainval_data = CustomImageDataset(traindatadir, 'annotations_train_calccluster_real.csv', 'colored_to_jpg', 'mask_to_png')

# testdatadir = 'B:\\Astrid\\Preprocessed\\test_set_preprocessedcalc'

testdatadir = r'B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs'
test_data = CustomImageDatasetMonai(testdatadir, 'annotations_test_real.csv', 'colored_to_jpg', 'mask_to_png')
# trainval_data = CustomImageDatasetMonai(testdatadir, 'annotations_train_real.csv', 'colored_to_jpg', 'mask_to_png')

savedir = 'B:\\Astrid\\Preprocessed\\ModelsAll'


In [8]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import utils
from torch.utils.data import random_split

# Read test dataset
# train_dataloader = DataLoader(trainval_data, batch_size=4, shuffle=False, collate_fn=utils.collate_fn)
# test_dataloader = DataLoader(trainval_data, batch_size=4, shuffle=False, collate_fn=utils.collate_fn)
test_dataloader = DataLoader(test_data, batch_size=4, shuffle=False, collate_fn=utils.collate_fn)


In [9]:
len(test_data), len(trainval_data)

NameError: name 'trainval_data' is not defined

In [10]:
test_data.lenclasses()

(190, 214)

## Load model with mobilenet backbone

In [190]:
import torchvision
from torchvision.models.detection import FasterRCNN, MaskRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# anchor_generator = AnchorGenerator(sizes=((4, 8, 16, 32, 64, 128),),
#                                    aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

mask_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                     output_size=14,
                                                     sampling_ratio=2)


# put the pieces together inside a MaskRCNN model
model = MaskRCNN(backbone,
                 num_classes=3,
                 rpn_anchor_generator=anchor_generator,
                 box_roi_pool=roi_pooler,
                 mask_roi_pool=mask_roi_pooler,
                 min_size=400,
                 max_size=650)


## Load model with resnet backbone

In [11]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

num_classes = 3

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

mask_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                     output_size=14,
                                                     sampling_ratio=2)

# anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
#                                    aspect_ratios=((0.5, 1.0, 2.0),))

# load an instance segmentation model pre-trained on COCO
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True,
                                                           box_roi_pool=roi_pooler,
                                                           mask_roi_pool=mask_roi_pooler,
                                                           min_size=400,
                                                           max_size=650)

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


# now get the number of input features for the mask classifier
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
# and replace the mask predictor with a new one
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                   hidden_layer,
                                                   num_classes)

In [12]:
model.to(device)

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(400,), max_size=650, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(inp

In [13]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9)

In [14]:
## Load from dict
# model.load_state_dict(torch.load(os.path.join(savedir, 'dict_malsynth.dict')))

## Load from checkpoint
checkpoint = torch.load(os.path.join('B:\\Astrid\\Preprocessed\\ModelsAll', 'model103_Monai_resnet_benmal_smalllr_allcalcssynth_50.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
# loss = checkpoint['loss']

model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(400,), max_size=650, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(inp

In [15]:
iter_data = iter(test_dataloader)

already_used = 0
used_now = 0
while used_now < already_used :
    test_images, test_targets = next(iter_data)
    used_now += 1

In [74]:
# For Testing
test_images, test_targets = next(iter_data)
test_image_list = list(image for image in test_images)
test_target_list = [{k: v for k, v in t.items()} for t in test_targets]

patient 4 B:\Astrid\Preprocessed\220615_preprocessed\realCalcs_prepCalcs\colored_to_jpg\MUMC_0160_R_CC.jpg
patient 5 B:\Astrid\Preprocessed\220615_preprocessed\realCalcs_prepCalcs\colored_to_jpg\MUMC_0227_R_CC.jpg
patient 6 B:\Astrid\Preprocessed\220615_preprocessed\realCalcs_prepCalcs\colored_to_jpg\MUMC_0258_R_CC.jpg
patient 7 B:\Astrid\Preprocessed\220615_preprocessed\realCalcs_prepCalcs\colored_to_jpg\MUMC_0302_R_CC.jpg


In [83]:
t

{'image_id': tensor([7]),
 'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8),
 'boxes': tensor([[166., 189., 195., 237.]]),
 'area': tensor([1392.]),
 'labels': tensor([2]),
 'iscrowd': tensor([0, 0])}

In [61]:
# test_images_cuda = []
# for te in test_images:
#     test_images_cuda.append(te.to(device))
    
test_images_cuda = [to.to(device) for te in test_images]

In [62]:
predictions = model(test_images_cuda)

In [134]:
predictions

[{'boxes': tensor([[132.8235, 296.8828, 170.9064, 339.3849],
          [130.6877, 296.5253, 173.2211, 339.0659]], device='cuda:0',
         grad_fn=<StackBackward0>),
  'labels': tensor([2, 1], device='cuda:0'),
  'scores': tensor([0.6075, 0.3531], device='cuda:0', grad_fn=<IndexBackward0>),
  'masks': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]]],
  
  
          [[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0',
         grad_fn=<UnsqueezeBackward0>)},
 {'boxes': tensor([[144.7954, 294.9714, 186.6304, 310.7547],
      

In [128]:


save_pred = nms(predictions[1]['boxes'], predictions[1]['scores'], 0.2)

save_pred

tensor([0, 1], device='cuda:0')

In [140]:
len(save_pred)

0

In [141]:
# predictions_nms = dict.fromkeys(predictions[1], )
from collections import defaultdict

predictions_nms = []

for pred_idx in range(4) :
    save_pred = nms(predictions[pred_idx]['boxes'], predictions[pred_idx]['scores'], 0.2)
    
    predictions_nms_idx = defaultdict(list)

    for idx in range(len(predictions[pred_idx]['boxes'])) :
        if idx in save_pred :
            print(idx)
            for key in predictions[pred_idx].keys() :
                predictions_nms_idx[key].append(predictions[pred_idx][key][idx])
                
    if len(save_pred) == 0 :
            print('Zero length')
            predictions_nms_idx = predictions[pred_idx].copy()

    predictions_nms.append(predictions_nms_idx)


0
0
1
Zero length
Zero length


In [142]:
predictions_nms

[defaultdict(list,
             {'boxes': [tensor([132.8235, 296.8828, 170.9064, 339.3849], device='cuda:0',
                      grad_fn=<SelectBackward0>)],
              'labels': [tensor(2, device='cuda:0')],
              'scores': [tensor(0.6075, device='cuda:0', grad_fn=<SelectBackward0>)],
              'masks': [tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        ...,
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.],
                        [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0',
                      grad_fn=<SelectBackward0>)]}),
 defaultdict(list,
             {'boxes': [tensor([144.7954, 294.9714, 186.6304, 310.7547], device='cuda:0',
                      grad_fn=<SelectBackward0>),
               tensor([ 72.3389, 133.4943, 115.2483, 163.3219], device='cuda:0',
              

In [64]:
test_target_list

[{'image_id': tensor([0]),
  'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8),
  'boxes': tensor([[126., 291., 187., 347.]]),
  'area': tensor([3416.]),
  'labels': tensor([1]),
  'iscrowd': tensor([0, 0])},
 {'image_id': tensor([1]),
  'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8),
  'boxes': tensor([[ 18., 148., 400., 397.]]),
  'area': tensor([95118.]),
  'labels': tensor([2]),
  'iscrowd': tensor([0, 0])},
 {'image_id': tensor([2]),
  'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
  

In [13]:
device_cpu = torch.device('cpu')

In [14]:
for pred_idx, pred in enumerate(predictions) :
    
    target_box = test_target_list[pred_idx]['boxes']
    target_area = test_target_list[pred_idx]['area']
    target_id = test_target_list[pred_idx]['image_id']
    
    target_label = test_target_list[pred_idx]['labels']
    
    for box_idx, pred_box in enumerate(pred['boxes']) :
        pred_score = pred['scores'][box_idx]
        pred_area = (pred_box[2].to(device_cpu)-pred_box[0].to(device_cpu))*(pred_box[3].to(device_cpu)-pred_box[1].to(device_cpu))
            
#         print(box_idx, target_box, pred_box, pred_score)
        
        xs = max(target_box[0][0],pred_box[0].to(device_cpu))
        ys = max(target_box[0][1],pred_box[1].to(device_cpu))
        xe = min(target_box[0][2],pred_box[2].to(device_cpu))
        ye = min(target_box[0][3],pred_box[3].to(device_cpu))
        if xe-xs > 0 and ye-ys > 0 :
            intersection_area = (xe-xs)*(ye-ys)
        else :
            intersection_area = -1
            
        union_area = pred_area + target_area - intersection_area
        
        if pred_score > 0.25 and intersection_area > 0.1*union_area :
            print('Lesion detected', target_id, pred_score, intersection_area)
                       

        pred_label = pred['labels'][box_idx]
        
        if pred_label.to(device_cpu) == target_label :
            print('Lesion correctly classified', target_id, target_label)

NameError: name 'predictions' is not defined

In [199]:
import time

def evaluate_kulum(model, data_loader, device):
#     n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
#     torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
#     model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"

#     evaluator_time = time.time()
    
    detected_list = []
    for images, targets in metric_logger.log_every(data_loader, 10, header):
        images = list(img.to(device) for img in images)

#         if torch.cuda.is_available():
#             torch.cuda.synchronize()
#         model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
#         model_time = time.time() - model_time

#         evaluator_time = time.time() - evaluator_time
#         metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

        for pred_idx, pred in enumerate(outputs) :

            target_box = targets[pred_idx]['boxes']
            target_area = targets[pred_idx]['area']
            target_id = targets[pred_idx]['image_id']

            for box_idx, pred_box in enumerate(pred['boxes']) :
                pred_score = pred['scores'][box_idx]
                pred_area = (pred_box[2]-pred_box[0])*(pred_box[3]-pred_box[1])

                xs = max(target_box[0][0],pred_box[0])
                ys = max(target_box[0][1],pred_box[1])
                xe = min(target_box[0][2],pred_box[2])
                ye = min(target_box[0][3],pred_box[3])
                if xe-xs > 0 and ye-ys > 0 :
                    intersection_area = (xe-xs)*(ye-ys)
                else :
                    intersection_area = -1

                union_area = target_area + pred_area - intersection_area
                if pred_score > 0.25 and intersection_area > 0.1*union_area :
                    print('Lesion detected', target_id, pred_score, intersection_area)
                    detected_list.append([target_id, pred_score, intersection_area])
                    
        del images, targets, outputs, pred
        torch.cuda.empty_cache()   

    # gather the stats from all processes
#     metric_logger.synchronize_between_processes()
#     print("Averaged stats:", metric_logger)

#     torch.set_num_threads(n_threads)
    
    return metric_logger, detected_list

In [16]:
def evaluate_kulum2(model, data_loader, device):
    model.eval()
    
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"
    
    iter_data = iter(data_loader)
    
    # Lists with all detected regions
    detected_correctben = []
    detected_correctmal = []
    detected_wrongbenpred = []
    detected_wrongmalpred = []
    detected_falseben = []
    detected_falsemal = []
    # Lesions with cases and what was detected
    detected_ben = []
    detected_mal = []
    detected_wrongben = []
    detected_wrongmal = []
    notdetected_ben = []
    notdetected_mal = []
    
    for images, targets in metric_logger.log_every(data_loader, 10, header):
        
#     while iter_data :

#         images, targets = next(iter_data)
        image_list = list(image for image in images)
        target_list = [{k: v for k, v in t.items()} for t in targets]

        images_cuda = []
        for te in images :
            images_cuda.append(te.to(device))

        with torch.no_grad() :
            predictions_cuda = model(images_cuda)
        predictions = [{k: v.to('cpu') for k, v in t.items()} for t in predictions_cuda]

        for pred_idx, pred in enumerate(predictions) :

            target_box = targets[pred_idx]['boxes']
            target_area = targets[pred_idx]['area']
            target_id = targets[pred_idx]['image_id']
            target_label = targets[pred_idx]['labels']
            
            targetben_detectedben = 0
            targetmal_detectedmal = 0
            targetben_detectedmal = 0
            targetmal_detectedben = 0

            for box_idx, pred_box in enumerate(pred['boxes'][:5]) :
                pred_score = pred['scores'][box_idx]
                pred_area = (pred_box[2]-pred_box[0])*(pred_box[3]-pred_box[1])

                xs = max(target_box[0][0],pred_box[0])
                ys = max(target_box[0][1],pred_box[1])
                xe = min(target_box[0][2],pred_box[2])
                ye = min(target_box[0][3],pred_box[3])
                if xe-xs > 0 and ye-ys > 0 :
                    intersection_area = (xe-xs)*(ye-ys)
                else :
                    intersection_area = -1

                union_area = target_area + pred_area - intersection_area
                
                if pred_score > 0.25 :
                    if intersection_area>0.1*union_area and target_label==pred['labels'][box_idx] :
                        print('Detected, correct class', target_id, pred_score, target_label, pred['labels'][box_idx])
                        
                        if target_label == 1 :
                            # A benign lesion is correctly detected as bening
                            detected_correctben.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                            targetben_detectedben = 1
                        else :
                            # A malignant lesion is correctly detected as malignant
                            detected_correctmal.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                            targetmal_detectedmal = 1
                    
                    elif intersection_area>0.1*union_area :
                        print('Detected, wrong class', target_id, pred_score, target_label, pred['labels'][box_idx])
                        
                        if pred['labels'][box_idx] == 1 :
                            # A malignant lesion is wrongly detected as benign
                            detected_wrongbenpred.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]]) 
                            targetmal_detectedben = 1
                        else :
                            # A benign lesion is wrongly detected as malignant
                            detected_wrongmalpred.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                            targetben_detectedmal = 1
                        
                    elif pred['labels'][box_idx]==1 :
                        print('False detected benign')
                        detected_falseben.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                    elif pred['labels'][box_idx]==2 :
                        print('False detected malignant')
                        detected_falsemal.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                        
            if targetben_detectedben > 0 :
                detected_ben.append([target_id])
            elif targetmal_detectedmal > 0 :
                detected_mal.append([target_id])
            elif targetben_detectedmal > 0 :
                detected_wrongben.append([target_id])
            elif targetmal_detectedben > 0 :
                detected_wrongmal.append([target_id])
            elif target_label == 1 :
                notdetected_ben.append([target_id])
            else :
                notdetected_mal.append(target_id)
                    
        del images_cuda, predictions_cuda
        torch.cuda.empty_cache()   
    
    return metric_logger, detected_correctben, detected_correctmal, detected_wrongbenpred, detected_wrongmalpred, detected_falseben, detected_falsemal, detected_ben, detected_mal, detected_wrongben, detected_wrongmal,notdetected_ben, notdetected_mal

In [17]:
ml, dcb, dcm, dwbp, dwmp, dfb, dfm, db, dm, dwb, dwm, ndb, ndm = evaluate_kulum2(model, test_dataloader, device)


patient 0 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0014_R_CC.jpg


C:\Users\P70074720\.conda\envs\pytorch36\lib\site-packages\torch\nn\functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


patient 1 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0019_L_CC.jpg
patient 2 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0025_L_CC.jpg
patient 3 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0037_L_CC.jpg


C:\Users\P70074720\.conda\envs\pytorch36\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


False detected malignant
Detected, correct class tensor([0]) tensor(0.8922) tensor([1]) tensor(1)
Detected, wrong class tensor([1]) tensor(0.7888) tensor([1]) tensor(2)
False detected malignant
False detected malignant
False detected benign
Detected, correct class tensor([1]) tensor(0.4090) tensor([1]) tensor(1)
False detected benign
Detected, correct class tensor([3]) tensor(0.9781) tensor([2]) tensor(2)
Test:  [  0/101]  eta: 0:12:40    time: 7.5277  data: 1.8878  max mem: 1136
patient 4 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0049_L_CC.jpg
patient 5 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0055_L_CC.jpg
patient 6 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0063_R_CC.jpg
patient 7 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0071_L_CC.jpg
Detected, correct class tensor([4]) tensor(0.9221) tensor([2]) tensor(2)
False detected maligna

...

patient 184 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1704_R_CC.jpg
patient 185 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1707_L_CC.jpg
patient 186 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1708_L_CC.jpg
patient 187 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1710_L_CC.jpg
Detected, correct class tensor([184]) tensor(0.9914) tensor([1]) tensor(1)
Detected, correct class tensor([185]) tensor(0.5699) tensor([2]) tensor(2)
Detected, correct class tensor([186]) tensor(0.9961) tensor([2]) tensor(2)
False detected benign
patient 188 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1713_R_CC.jpg
patient 189 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1722_R_CC.jpg
patient 190 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1728_L_CC.jpg
pati

Detected, correct class tensor([228]) tensor(0.9519) tensor([1]) tensor(1)
Detected, correct class tensor([229]) tensor(0.9268) tensor([2]) tensor(2)
Detected, correct class tensor([229]) tensor(0.2959) tensor([2]) tensor(2)
Detected, correct class tensor([230]) tensor(0.9863) tensor([2]) tensor(2)
Detected, correct class tensor([231]) tensor(0.9843) tensor([2]) tensor(2)
patient 232 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0302_R_MLO.jpg
patient 233 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0316_R_MLO.jpg
patient 234 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0326_R_MLO.jpg
patient 235 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0333_L_MLO.jpg
Detected, correct class tensor([232]) tensor(0.8304) tensor([2]) tensor(2)
Detected, correct class tensor([233]) tensor(0.9259) tensor([2]) tensor(2)
Detected, correct class tensor([234]) tenso

patient 278 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0627_R_MLO.jpg
patient 279 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0630_R_MLO.jpg
Detected, correct class tensor([276]) tensor(0.9217) tensor([1]) tensor(1)
False detected malignant
Detected, correct class tensor([277]) tensor(0.9977) tensor([2]) tensor(2)
Detected, correct class tensor([278]) tensor(0.9092) tensor([2]) tensor(2)
Detected, correct class tensor([279]) tensor(0.9956) tensor([1]) tensor(1)
patient 280 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0644_R_MLO.jpg
patient 281 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0646_R_MLO.jpg
patient 282 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0647_L_MLO.jpg
patient 283 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0680_L_MLO.jpg
Detected, correct class ten

patient 322 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1027_R_MLO.jpg
patient 323 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1028_L_MLO.jpg
Detected, wrong class tensor([320]) tensor(0.4020) tensor([2]) tensor(1)
Detected, wrong class tensor([321]) tensor(0.9433) tensor([2]) tensor(1)
Detected, wrong class tensor([322]) tensor(0.7172) tensor([2]) tensor(1)
Detected, correct class tensor([322]) tensor(0.5853) tensor([2]) tensor(2)
Detected, correct class tensor([323]) tensor(0.6279) tensor([2]) tensor(2)
False detected malignant
Detected, correct class tensor([323]) tensor(0.4837) tensor([2]) tensor(2)
False detected benign
Test:  [ 80/101]  eta: 0:01:04    time: 3.3463  data: 2.6164  max mem: 1136
patient 324 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1040_L_MLO.jpg
patient 325 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1041_L_MLO.jpg
p

patient 364 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1431_L_MLO.jpg
patient 365 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1439_R_MLO.jpg
patient 366 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1451_L_MLO.jpg
patient 367 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1452_R_MLO.jpg
Detected, correct class tensor([365]) tensor(0.9012) tensor([1]) tensor(1)
False detected malignant
Detected, correct class tensor([366]) tensor(0.9438) tensor([1]) tensor(1)
Detected, correct class tensor([367]) tensor(0.9989) tensor([2]) tensor(2)
False detected malignant
patient 368 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1474_R_MLO.jpg
patient 369 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1482_R_MLO.jpg
patient 370 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colo

In [18]:
len(dcb), len(dcm), len(dwbp), len(dwmp), len(dfb), len(dfm)

(116, 174, 22, 21, 97, 105)

In [19]:
len(db), len(dm), len(dwb), len(dwm), len(ndb), len(ndm)

(116, 157, 11, 11, 63, 46)

In [20]:
def evaluate_kulum3(model, data_loader, device, conf=0.25, iou=0.1):
    model.eval()
    
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"
    
    iter_data = iter(data_loader)
    
    # Lists with all detected regions
    detected_correctben = []
    detected_correctmal = []
    detected_wrongbenpred = []
    detected_wrongmalpred = []
    detected_falseben = []
    detected_falsemal = []
    # Lesions with cases and what was detected
    detected_ben = []
    detected_mal = []
    detected_wrongben = []
    detected_wrongmal = []
    notdetected_ben = []
    notdetected_mal = []
    
    for images, targets in metric_logger.log_every(data_loader, 10, header):
        
        # Read in image list
        image_list = list(image for image in images)
        target_list = [{k: v for k, v in t.items()} for t in targets]

        images_cuda = []
        for te in images :
            images_cuda.append(te.to(device))

        # Make predictions
        with torch.no_grad() :
            predictions_cuda = model(images_cuda)
        predictions = [{k: v.to('cpu') for k, v in t.items()} for t in predictions_cuda]
        
        # Eliminate overlapping predictions with non-max suppression
        predictions_nms = []
        for pred_idx in range(len(predictions)) :
            save_pred = nms(predictions[pred_idx]['boxes'], predictions[pred_idx]['scores'], 0.2)

            predictions_nms_idx = defaultdict(list)

            for idx in range(len(predictions[pred_idx]['boxes'])) :
                if idx in save_pred :
#                     print(idx)
                    for key in predictions[pred_idx].keys() :
                        predictions_nms_idx[key].append(predictions[pred_idx][key][idx])

            if len(save_pred) == 0 :
#                     print('Zero length')
                    predictions_nms_idx = predictions[pred_idx].copy()

            predictions_nms.append(predictions_nms_idx)
        
        # Check for each predicted box whether it is false or true positive
        for pred_idx, pred in enumerate(predictions_nms) :

            target_box = targets[pred_idx]['boxes']
            target_area = targets[pred_idx]['area']
            target_id = targets[pred_idx]['image_id']
            target_label = targets[pred_idx]['labels']
            
            targetben_detectedben = 0
            targetmal_detectedmal = 0
            targetben_detectedmal = 0
            targetmal_detectedben = 0

            # Only check 5 boxes with highest scores
            for box_idx, pred_box in enumerate(pred['boxes'][:5]) :
                pred_score = pred['scores'][box_idx]
                pred_area = (pred_box[2]-pred_box[0])*(pred_box[3]-pred_box[1])

                xs = max(target_box[0][0],pred_box[0])
                ys = max(target_box[0][1],pred_box[1])
                xe = min(target_box[0][2],pred_box[2])
                ye = min(target_box[0][3],pred_box[3])
                if xe-xs > 0 and ye-ys > 0 :
                    intersection_area = (xe-xs)*(ye-ys)
                else :
                    intersection_area = -1
                union_area = target_area + pred_area - intersection_area
                
                # Only consider predicted boxes with high score
                if pred_score > conf :
                    # If IoU is larger that threshold and correct class -- true positive
                    if intersection_area>iou*union_area and target_label==pred['labels'][box_idx] :
                        print('Detected, correct class', target_id, pred_score, target_label, pred['labels'][box_idx])
                        
                        if target_label == 1 :
                            # A benign lesion is correctly detected as bening
                            detected_correctben.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                            targetben_detectedben = 1
                        else :
                            # A malignant lesion is correctly detected as malignant
                            detected_correctmal.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                            targetmal_detectedmal = 1
                    
                    # If IoU is larger than threshold and wrong class -- false positive
                    elif intersection_area>0.1*union_area :
                        print('Detected, wrong class', target_id, pred_score, target_label, pred['labels'][box_idx])
                        
                        if pred['labels'][box_idx] == 1 :
                            # A malignant lesion is wrongly detected as benign
                            detected_wrongbenpred.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]]) 
                            targetmal_detectedben = 1
                        else :
                            # A benign lesion is wrongly detected as malignant
                            detected_wrongmalpred.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                            targetben_detectedmal = 1
                    
                    # If IoU smaller than threshold -- false positive
                    elif pred['labels'][box_idx]==1 :
                        print('False detected benign')
                        detected_falseben.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                    elif pred['labels'][box_idx]==2 :
                        print('False detected malignant')
                        detected_falsemal.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                        
            if targetben_detectedben > 0 :
                detected_ben.append([target_id]) # part of true positive
            elif targetmal_detectedmal > 0 :
                detected_mal.append([target_id]) # part of true positive
            elif targetben_detectedmal > 0 :
                detected_wrongben.append([target_id]) # part of false positive
            elif targetmal_detectedben > 0 :
                detected_wrongmal.append([target_id]) # part of false positive
            elif target_label == 1 :
                notdetected_ben.append([target_id]) # part of false negative
            else :
                notdetected_mal.append(target_id) # part of false negative
                    
        del images_cuda, predictions_cuda
        torch.cuda.empty_cache()   
        
    if len(detected_ben)+len(detected_mal)+len(detected_falseben)+len(detected_falsemal)+len(detected_wrongben)+len(detected_wrongmal) > 0:
        precision_all = (len(detected_ben)+len(detected_mal))/(len(detected_ben)+len(detected_mal)+len(detected_falseben)+len(detected_falsemal)+len(detected_wrongben)+len(detected_wrongmal))
    else :
        precision_all = 0
    if len(detected_ben)+len(detected_falseben)+len(detected_wrongmal) > 0 :
        precision_ben = len(detected_ben)/(len(detected_ben)+len(detected_falseben)+len(detected_wrongmal))
    else :
        precision_ben = 0
    if len(detected_mal)+len(detected_falsemal)+len(detected_wrongben) > 0 :
        precision_mal = len(detected_mal)/(len(detected_mal)+len(detected_falsemal)+len(detected_wrongben))
    else :
        precision_mal = 0
    precision_avg = precision_ben*(len(detected_ben)+len(notdetected_ben))+precision_mal*(len(detected_mal)+len(notdetected_mal))
    precision_avg /= len(detected_ben)+len(notdetected_ben)+len(detected_mal)+len(notdetected_mal)

    recall_all = (len(detected_ben)+len(detected_mal))/(len(detected_ben)+len(notdetected_ben)+len(detected_mal)+len(notdetected_mal))
    recall_ben = len(detected_ben)/(len(detected_ben)+len(notdetected_ben))
    recall_mal = len(detected_mal)/(len(detected_mal)+len(notdetected_mal))
    recall_avg = recall_ben*(len(detected_ben)+len(notdetected_ben))+recall_mal*(len(detected_mal)+len(notdetected_mal))
    recall_avg /= len(detected_ben)+len(notdetected_ben)+len(detected_mal)+len(notdetected_mal)

    print('Precision', precision_all, precision_ben, precision_mal, precision_avg)
    print('Recall', recall_all, recall_ben, recall_mal, recall_avg)
        
    return precision_ben, precision_mal, precision_avg, recall_ben, recall_mal, recall_avg
#     return metric_logger, detected_correctben, detected_correctmal, detected_wrongbenpred, detected_wrongmalpred, detected_falseben, detected_falsemal, detected_ben, detected_mal, detected_wrongben, detected_wrongmal,notdetected_ben, notdetected_mal

In [ ]:
prec_ben_list, prec_mal_list, prec_list = [], [], []
recall_ben_list, recall_mal_list, recall_list = [], [], []

for conf in np.arange(1.0,-0.1,-0.05) :
    print(conf)
    precb, precm, prec, recallb, recallm, recall = evaluate_kulum3(model, test_dataloader, device, conf=conf)
    prec_ben_list.append(precb)
    prec_mal_list.append(precm)
    prec_list.append(prec)
    recall_ben_list.append(recallb)
    recall_mal_list.append(recallm)
    recall_list.append(recall)

1.0
patient 0 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0014_R_CC.jpg
patient 1 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0019_L_CC.jpg
patient 2 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0025_L_CC.jpg
patient 3 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0037_L_CC.jpg
Test:  [  0/101]  eta: 0:03:59    time: 2.3703  data: 1.6984  max mem: 1136
patient 4 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0049_L_CC.jpg
patient 5 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0055_L_CC.jpg
patient 6 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0063_R_CC.jpg
patient 7 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0071_L_CC.jpg
patient 8 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_t

patient 76 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0614_R_CC.jpg
patient 77 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0627_R_CC.jpg
patient 78 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0630_R_CC.jpg
patient 79 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0644_R_CC.jpg
patient 80 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0646_R_CC.jpg
patient 81 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0647_L_CC.jpg
patient 82 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0680_L_CC.jpg
patient 83 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0689_L_CC.jpg
Test:  [ 20/101]  eta: 0:03:42    time: 2.7714  data: 1.9793  max mem: 1136
patient 84 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colo

patient 152 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1314_R_CC.jpg
patient 153 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1317_L_CC.jpg
patient 154 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1327_R_CC.jpg
patient 155 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1329_R_CC.jpg
patient 156 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1375_R_CC.jpg
patient 157 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1393_L_CC.jpg
patient 158 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1395_L_CC.jpg
patient 159 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1396_L_CC.jpg
patient 160 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1400_L_CC.jpg
patient 161 B:\Astrid\Preprocessed\22

patient 227 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0258_R_MLO.jpg
patient 228 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0263_R_MLO.jpg
patient 229 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0277_R_MLO.jpg
patient 230 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0284_R_MLO.jpg
patient 231 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0296_R_MLO.jpg
patient 232 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0302_R_MLO.jpg
patient 233 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0316_R_MLO.jpg
patient 234 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0326_R_MLO.jpg
patient 235 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0333_L_MLO.jpg
patient 236 B:\Astrid\Prepro

patient 302 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0830_L_MLO.jpg
patient 303 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0831_L_MLO.jpg
patient 304 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0843_L_MLO.jpg
patient 305 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0871_L_MLO.jpg
patient 306 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0873_L_MLO.jpg
patient 307 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0877_R_MLO.jpg
patient 308 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0883_R_MLO.jpg
patient 309 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0886_L_MLO.jpg
patient 310 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0905_R_MLO.jpg
patient 311 B:\Astrid\Prepro

...

patient 98 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0799_L_CC.jpg
patient 99 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0803_L_CC.jpg
Detected, correct class tensor([97]) tensor(0.9676) tensor([1]) tensor(1)
patient 100 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0811_R_CC.jpg
patient 101 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0815_L_CC.jpg
patient 102 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0830_L_CC.jpg
patient 103 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0831_L_CC.jpg
Detected, correct class tensor([100]) tensor(0.9810) tensor([1]) tensor(1)
Detected, correct class tensor([102]) tensor(0.9916) tensor([1]) tensor(1)
Detected, correct class tensor([103]) tensor(0.9962) tensor([2]) tensor(2)
patient 104 B:\Astrid\Preprocessed\220615_preprocessed\realAl

patient 155 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1329_R_CC.jpg
Detected, correct class tensor([152]) tensor(0.9979) tensor([2]) tensor(2)
Detected, correct class tensor([155]) tensor(0.9626) tensor([1]) tensor(1)
patient 156 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1375_R_CC.jpg
patient 157 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1393_L_CC.jpg
patient 158 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1395_L_CC.jpg
patient 159 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1396_L_CC.jpg
False detected benign
patient 160 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1400_L_CC.jpg
patient 161 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1400_R_CC.jpg
patient 162 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colore

patient 213 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0081_L_MLO.jpg
patient 214 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0084_L_MLO.jpg
patient 215 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0112_R_MLO.jpg
Detected, correct class tensor([212]) tensor(0.9972) tensor([2]) tensor(2)
Detected, correct class tensor([213]) tensor(0.9902) tensor([2]) tensor(2)
patient 216 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0119_R_MLO.jpg
patient 217 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0123_R_MLO.jpg
patient 218 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0129_R_MLO.jpg
patient 219 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0147_L_MLO.jpg
False detected malignant
Detected, correct class tensor([218]) tensor(0.9990) tensor([1]) tens

patient 271 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0577_L_MLO.jpg
patient 272 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0578_L_MLO.jpg
patient 273 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0586_L_MLO.jpg
patient 274 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0591_L_MLO.jpg
patient 275 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0594_L_MLO.jpg
Detected, correct class tensor([272]) tensor(0.9928) tensor([1]) tensor(1)
Detected, correct class tensor([274]) tensor(0.9914) tensor([2]) tensor(2)
Detected, correct class tensor([275]) tensor(0.9907) tensor([1]) tensor(1)
patient 276 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0611_L_MLO.jpg
patient 277 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_0614_R_MLO.jpg
patient 278 B:\Astr

patient 329 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1060_R_MLO.jpg
patient 330 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1072_L_MLO.jpg
patient 331 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1077_L_MLO.jpg
Detected, correct class tensor([329]) tensor(0.9528) tensor([1]) tensor(1)
Detected, correct class tensor([331]) tensor(0.9948) tensor([1]) tensor(1)
patient 332 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1079_R_MLO.jpg
patient 333 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1102_L_MLO.jpg
patient 334 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1120_R_MLO.jpg
patient 335 B:\Astrid\Preprocessed\220615_preprocessed\realAll_prepCalcs\colored_to_jpg\MUMC_1123_L_MLO.jpg
Detected, correct class tensor([333]) tensor(0.9938) tensor([2]) tensor(2)
False detected beni

In [ ]:
prec_list, recall_list

In [ ]:
plt.scatter(recall_list, prec_list)
plt.scatter(recall_ben_list, prec_ben_list)
plt.scatter(recall_mal_list, prec_mal_list)

In [ ]:
def average_precision(prec_list, recall_list) :
    avg_prec = 0
    for k in range(1, len(prec_list)) :
        avg_prec += (recall_list[k]-recall_list[k-1])*max(prec_list[k],prec_list[k-1])
        print('Diff recall', recall_list[k]-recall_list[k-1], 'Max precision', max(prec_list[k],prec_list[k-1]))
        
    return avg_prec

In [ ]:
average_precision(prec_ben_list, recall_ben_list)

In [ ]:
average_precision(prec_mal_list, recall_mal_list)

In [ ]:
def evaluate_kulum2(model, data_loader, device):
    model.eval()
    
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"
    
    iter_data = iter(data_loader)
    
    # Lists with all detected regions
    detected_correctben = []
    detected_correctmal = []
    detected_wrongbenpred = []
    detected_wrongmalpred = []
    detected_falseben = []
    detected_falsemal = []
    # Lesions with cases and what was detected
    detected_ben = []
    detected_mal = []
    detected_wrongben = []
    detected_wrongmal = []
    notdetected_ben = []
    notdetected_mal = []
    
    for images, targets in metric_logger.log_every(data_loader, 10, header):
        
#     while iter_data :

#         images, targets = next(iter_data)
        image_list = list(image for image in images)
        target_list = [{k: v for k, v in t.items()} for t in targets]

        images_cuda = []
        for te in images :
            images_cuda.append(te.to(device))

        with torch.no_grad() :
            predictions_cuda = model(images_cuda)
        predictions = [{k: v.to('cpu') for k, v in t.items()} for t in predictions_cuda]

        for pred_idx, pred in enumerate(predictions) :

            target_box = targets[pred_idx]['boxes']
            target_area = targets[pred_idx]['area']
            target_id = targets[pred_idx]['image_id']
            target_label = targets[pred_idx]['labels']
            
            targetben_detectedben = 0
            targetmal_detectedmal = 0
            targetben_detectedmal = 0
            targetmal_detectedben = 0

            for box_idx, pred_box in enumerate(pred['boxes'][:5]) :
                pred_score = pred['scores'][box_idx]
                pred_area = (pred_box[2]-pred_box[0])*(pred_box[3]-pred_box[1])

                xs = max(target_box[0][0],pred_box[0])
                ys = max(target_box[0][1],pred_box[1])
                xe = min(target_box[0][2],pred_box[2])
                ye = min(target_box[0][3],pred_box[3])
                if xe-xs > 0 and ye-ys > 0 :
                    intersection_area = (xe-xs)*(ye-ys)
                else :
                    intersection_area = -1

                union_area = target_area + pred_area - intersection_area
                
                if pred_score > 0.25 :
                    if intersection_area>0.1*union_area and target_label==pred['labels'][box_idx] :
                        print('Detected, correct class', target_id, pred_score, target_label, pred['labels'][box_idx])
                        
                        if target_label == 1 :
                            # A benign lesion is correctly detected as bening
                            detected_correctben.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                            targetben_detectedben = 1
                        else :
                            # A malignant lesion is correctly detected as malignant
                            detected_correctmal.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                            targetmal_detectedmal = 1
                    
                    elif intersection_area>0.1*union_area :
                        print('Detected, wrong class', target_id, pred_score, target_label, pred['labels'][box_idx])
                        
                        if pred['labels'][box_idx] == 1 :
                            # A malignant lesion is wrongly detected as benign
                            detected_wrongbenpred.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]]) 
                            targetmal_detectedben = 1
                        else :
                            # A benign lesion is wrongly detected as malignant
                            detected_wrongmalpred.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                            targetben_detectedmal = 1
                        
                    elif pred['labels'][box_idx]==1 :
                        print('False detected benign')
                        detected_falseben.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                    elif pred['labels'][box_idx]==2 :
                        print('False detected malignant')
                        detected_falsemal.append([target_id, pred_score, pred['boxes'][box_idx], pred['labels'][box_idx]])
                        
            if targetben_detectedben > 0 :
                detected_ben.append([target_id])
            elif targetmal_detectedmal > 0 :
                detected_mal.append([target_id])
            elif targetben_detectedmal > 0 :
                detected_wrongben.append([target_id])
            elif targetmal_detectedben > 0 :
                detected_wrongmal.append([target_id])
            elif target_label == 1 :
                notdetected_ben.append([target_id])
            else :
                notdetected_mal.append(target_id)
                    
        del images_cuda, predictions_cuda
        torch.cuda.empty_cache()   
    
    return metric_logger, detected_correctben, detected_correctmal, detected_wrongbenpred, detected_wrongmalpred, detected_falseben, detected_falsemal, detected_ben, detected_mal, detected_wrongben, detected_wrongmal,notdetected_ben, notdetected_mal

In [15]:
torch.cuda.empty_cache()

In [29]:
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset
import time

def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types

def evaluate(model, data_loader, device):
    
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    
    cpu_device = torch.device("cpu")
    
    model.eval()
    
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

#     detected_list = []
    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)
        
        for pred_idx, pred in enumerate(outputs) :

            target_box = targets[pred_idx]['boxes']
            target_area = targets[pred_idx]['area']
            target_id = targets[pred_idx]['image_id']
            target_name = targets[pred_idx]['image_name']

            for box_idx, pred_box in enumerate(pred['boxes']) :
                pred_score = pred['scores'][box_idx]
                pred_area = (pred_box[2]-pred_box[0])*(pred_box[3]-pred_box[1])

                xs = max(target_box[0][0],pred_box[0])
                ys = max(target_box[0][1],pred_box[1])
                xe = min(target_box[0][2],pred_box[2])
                ye = min(target_box[0][3],pred_box[3])
                if xe-xs > 0 and ye-ys > 0 :
                    overlap_area = (xe-xs)*(ye-ys)
                else :
                    overlap_area = -1

                if pred_score > 0.1 and overlap_area > 0.1*target_area :
                    print('Lesion detected', target_id, target_name, pred_score, overlap_area)
#                     detected_list.append([target_id, pred_score, overlap_area])
        

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [30]:
cocoev = evaluate(model, test_dataloader, device=device)

creating index...
index created!
Test:  [0/4]  eta: 0:00:00  model_time: 0.2190 (0.2190)  evaluator_time: 0.0190 (0.0190)  time: 0.2480  data: 0.0020  max mem: 7545
Test:  [3/4]  eta: 0:00:00  model_time: 0.1830 (0.1907)  evaluator_time: 0.0060 (0.0145)  time: 0.2145  data: 0.0010  max mem: 7717
Test: Total time: 0:00:00 (0.2150 s / it)
Averaged stats: model_time: 0.1830 (0.1907)  evaluator_time: 0.0060 (0.0145)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets